# Decorators

Decorators are a feature that many dynamic languages use to enable you to do more with the functions you write. This lesson is a tutorial to get you started with writing and using decorators.

## Expressions and Symbols

If we examine this line of code:

    three = 3

`3` is an expression (a very simple one), and `three` is a symbol. The result of that line of code is the symbol `three` is given the result of the expression `3`, which is happens to be the integer value 3.

Similarly we can examine these lines of code:

In [1]:
def three():
    return 3

Everything after the `:` character until lines are no longer indented is a *code block*; in this case the code block is the single line of code `return 3`. `three` is a symbol (just like it was above, and `def <symbol>(<arguments>):` is the syntax for defining a function. The result of executing that whole thing is that a function object is created and it is then assigned to the symbol `three`.

## Function Objects

So what is a function object? In Python, [everything is an object](http://www.diveintopython.net/getting_to_know_python/everything_is_an_object.html). A **function object** is a kind of object that *does* something; specifically a function object executes a series of statements and returns a value (which is, of course, another object).

In [2]:
type(three)

function

In [3]:
dir(three)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

Because function objects are *just* another kind of object, they can be assigned to variables, passed as parameters to functions, introspected, etc. Function objects are also called **callables**.

One thing you can do with objects is *extend* them. This is usually thought about with classes, where you define a class that extends another class to add an extra function, but any object can be extended. This is done using *decorators*.

## Decorators

A **decorator** is basically a wrapper around a callable that extends or enhances that callable in some way. There are many reasons to decorate, but the two most common are probably:

* registering a function with a system that consumes it in some way
* enhancing a function by adding extra code before or after it executes

### Functions That Return Functions

Because a decorator is a wrapper around another function (so it's a function), you write it the same way you write any other function:

In [4]:
def print_hi_first(func):
    def inner():
        print('hi')
        return func()
    return inner

This function is returning a function that does something when called: first it prints "hi", then it runs the function that was passed to it. It can be used successfully with any function that takes no arguments (because it calls that function with no arguments).

If we had a simple function like this:

In [5]:
def print_three():
    print('three')

print_three()

three


... we could wrap it in our extra function like this:

In [6]:
wrapped_print_three = print_hi_first(print_three)
wrapped_print_three()

hi
three


Calling the wrapped function does first what our decorator added, then it calls the inner function.

### Syntax

To make this easier, we can wrap our function using the decorator syntax:

In [7]:
@print_hi_first
def print_three():
    print('three')

print_three()

hi
three


Note that now the returned function is assigned to the name of our function... `print_three` is the new function that wraps `print_hi_first` around `print_three`. This is handy, because now any method using our `print_three` function simply gets the enhanced functionality without having to know that the function is decorated in any way or calling it in any special way.

### Function Parameters

Sometimes you need to decorate functions that take any arbitrary arguments, but you don't know what the argument list will look like. Python has syntax for this as well, using the `*args, **kwargs` notation at the end of the function parameter list:

In [8]:
def show_my_args(x, *args, **kwargs):
    print(args)
    print(kwargs)

`args` is a tuple containing any argument passed as an expression that wasn't already listed in the parameter argument list for the function. If we execute `show_my_args(1, 2, 3*3)` we will get a tuple of two integers: `(2, 9)` (note that the `1` was assigned to the first parameter, `x`).

In [9]:
show_my_args(1, 2, 3*3)

(2, 9)
{}


`kwargs` is a dictionary of key:value pairs containing any named argument passed that wasn't already listed in the parameter argument list for the function. If we execute `show_my_args(x=1, y=2, z=3)` we will get a dictionary of two key:value pairs: `{'y': 2, 'z': 3}` (note that `x` was assigned to the parameter called `x`).

In [10]:
show_my_args(x=1, y=2, z=3)

()
{'y': 2, 'z': 3}


This is how a function like `print()` can take an arbitrary number of arguments without knowing in advance what they will be, it just iterates over `args` and consumes and displays them one at a time.

## Putting It All Together

Let's say we want to write a decorator that prints all the arguments passed to a function before executing that function. We can do that like this:

In [11]:
def print_args_and_result(func):
    def inner(*args, **kwargs):
        print('args:', args)
        print('kwargs:', kwargs)
        result = func(*args, **kwargs)
        print('result:', result)
        return result
    return inner

Now if we create two functions and use our decorator on them:

In [12]:
@print_args_and_result
def linear_equation(m, x, b):
    return m + x + b

@print_args_and_result
def sum(*args):
    total = 0
    for x in args:
        total += x
    return total

... we can see the values passed in and the return value of those methods every time they are called:

In [13]:
linear_equation(3, 5, 7)

args: (3, 5, 7)
kwargs: {}
result: 15


15

In [14]:
sum(1, 2, 3)

args: (1, 2, 3)
kwargs: {}
result: 6


6

### Python Helpers for Decorators

Looking again at our `print_three` function above:

    def print_three():
        print('three')

... let's modify it to have a docstring:

In [15]:
def print_three():
    '''return the word "three"'''
    print('three')

Note that there a re a lot of things we can introspect about this function:

In [16]:
print_three.__doc__

'return the word "three"'

In [17]:
print_three.__module__

'__main__'

... and a whole lot more. But if we wrap `print_three`:

In [18]:
def print_hi_first(func):
    def inner():
        print('hi')
        return func()
    return inner

@print_hi_first
def print_three():
    '''return the word "three"'''
    print('three')

print_three.__doc__

... now were' going to lose all that information, because the `print_three` symbol is no longer assigned to the inner function.

Luckily, the `functools` module has a solution for us: the `wraps` decorator:

In [19]:
from functools import wraps

def print_hi_first(func):
    @wraps(func)
    def inner():
        print('hi')
        return func()
    return inner

@print_hi_first
def print_three():
    '''return the word "three"'''
    print('three')

print_three.__doc__

'return the word "three"'

## Learning More

I mentioned a couple reasons to use decorators above, but there are many more. Here are some ideas for a decorator to write to practice:

* time how long a method takes to run
* count how many times a method is run; use it to see how many times some recursive functions (like factorial, breadth-first search, and depth-first search) call themselves recursively
* combine the first two to count how many times a function is called and accumulate the total time spent in a function for the duration of a program that calls that function many times
* use a decorator to make sure one of the named parameters, if it exists, is of a particular type

You should also read more about **closures** and Python's variable scoping rules, as there are  some things you should know about what you can and can't expect the behavior of certain kinds of variables to be inside a decorator.

## Reference

* [Everything Is an Object](http://www.diveintopython.net/getting_to_know_python/everything_is_an_object.html)
* [5 reasons you need to learn to write Python decorators](https://www.oreilly.com/ideas/5-reasons-you-need-to-learn-to-write-python-decorators)
* [Primer on Python Decorators](https://realpython.com/primer-on-python-decorators/)
* "Function Decorators and Closures", chapter 7 of [Fluent Python](http://shop.oreilly.com/product/0636920032519.do)